## WORKSHOP HOMEWORK

### Question 1
What is the sum of the outputs of the generator for limit = 5?
What is the 13th number yielded?

In [4]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 5
generator = square_root_generator(limit)
sum_limit_5 = 0

for sqrt_value in generator:
    sum_limit_5 += sqrt_value
    # print(sqrt_value)

print(sum_limit_5)

8.382332347441762


### Question 2
What is the 13th number generated?

In [5]:
limit = 13
generator = square_root_generator(limit)
sum_limit_5 = 0

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


### Question 3
Append the two generators, after appending calculate the sum of all ages

In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [15]:
import duckdb

conn = duckdb.connect()

create_table_sql = '''
CREATE TABLE people (
    ID INTEGER,
    Name VARCHAR,
    Age INTEGER,
    City VARCHAR
)
'''
conn.execute(create_table_sql)

In [36]:
# insert first group
insert_sql = 'INSERT INTO people (ID, Name, Age, City) VALUES (?, ?, ?, ?)'
for person in people_1():
    conn.execute(insert_sql, (person["ID"], person["Name"], person["Age"], person["City"]))

In [37]:
ages = conn.execute("SELECT SUM(Age) FROM people").fetchone()
print(ages[0])

140


In [38]:
# insert second group
insert_sql = 'INSERT INTO people (ID, Name, Age, City) VALUES (?, ?, ?, ?)'
for person in people_2():
    conn.execute(insert_sql, (person["ID"], person["Name"], person["Age"], person["City"]))

In [39]:
ages = conn.execute("SELECT SUM(Age) FROM people").fetchone()
print(ages[0])

353


### Question 4
Merge the two generators using the ID column and calculate the sum of all ages

In [40]:
conn.execute('''
CREATE TABLE people_merged (
    ID INTEGER PRIMARY KEY,
    Name VARCHAR,
    Age INTEGER,
    City VARCHAR,
    Occupation VARCHAR
)
''')

insert_sql = 'INSERT INTO people_merged (ID, Name, Age, City) VALUES (?, ?, ?, ?)'

for person in people_1():
    try:
        conn.execute(insert_sql, (person["ID"], person["Name"], person["Age"], person["City"]))
    except Exception as e:
        print(f"An error occurred: {e}")

upsert_sql = '''
INSERT INTO people_merged (ID, Name, Age, City, Occupation) VALUES (?, ?, ?, ?, ?)
ON CONFLICT (ID) DO UPDATE SET Name = EXCLUDED.Name, Age = EXCLUDED.Age, City = EXCLUDED.City, Occupation = EXCLUDED.Occupation
'''

for person in people_2():
    try:
        conn.execute(upsert_sql, (person["ID"], person["Name"], person["Age"], person["City"], person["Occupation"]))
    except Exception as e:
        print(f"An error occurred: {e}")

In [41]:
# Verify the total number of records
total_records = conn.execute("SELECT COUNT(*) FROM people_merged").fetchall()[0][0]
print(f"Total records: {total_records}")

# Verify the details for ID 3
id_3_details = conn.execute("SELECT * FROM people_merged WHERE ID = 3").fetchall()[0]
print(f"Details for ID 3: {id_3_details}")

Total records: 8
Details for ID 3: (3, 'Person_3', 33, 'City_B', 'Job_3')


In [42]:
ages = conn.execute("SELECT SUM(Age) FROM people_merged").fetchone()
print(ages[0])

266
